

# Introduction

The following is my attempt to use sentiment analysis on news articles to figure out whether they fall in the positive, negative or neutral category.

I used newsapi to get the news articles
and, beautiful soup and text blob, to analyse them. Also, pandas and numpy to manipulate the data.

In [2]:

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup



The following is the api call to get the news articles.

I used "Christmas" as the query and added a random time frame, this can be changed according to the needs.

In [58]:


from newsapi import NewsApiClient


newsapi = NewsApiClient(api_key='365c300b05f646fb90fee48c3ab38e77')



all_articles = newsapi.get_everything(q='Christmas',
                                      
                                      from_param='2022-12-01',
                                      to='2022-12-24', sources='the-hindu,the-times-of-india,google-news-in',
                                      domains='thehindu.com,timesofindia.indiatimes.com,news.google.com',
                                      language='en',
                                      sort_by='popularity',
                                      page=1)


df=pd.DataFrame(all_articles['articles'])


index = pd.Index(range(1, 101, 1))
df = df.set_index(index)
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content
1,"{'id': 'the-times-of-india', 'name': 'The Time...",Reuters,Glittering gold gives markets some Christmas c...,MSCI's broadest index of Asia-Pacific shares o...,https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96386750,widt...",2022-12-21T03:19:42Z,SINGAPORE: Asian stocks were trying get into a...
2,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,UK PM Rishi Sunak uses troops to tackle Christ...,There is a wave of winter strikes planned over...,https://economictimes.indiatimes.com/news/inte...,"https://img.etimg.com/thumb/msid-96325937,widt...",2022-12-18T19:23:17Z,British Prime Minister Rishi Sunak on Sunday h...
3,"{'id': 'the-times-of-india', 'name': 'The Time...",TNN,Slowing US inflation raises hopes for Santa Cl...,The rally in the global markets could spill ov...,https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96209413,widt...",2022-12-14T02:21:19Z,MUMBAI: An inflation reading in the US that ca...
4,"{'id': 'the-times-of-india', 'name': 'The Time...",ET CONTRIBUTORS,Rising Covid cases is polishing gold’s appeal ...,The economic data during the week from the US ...,https://economictimes.indiatimes.com/markets/c...,"https://img.etimg.com/thumb/msid-96477641,widt...",2022-12-24T10:17:12Z,Gold prices reached a six-month high of $1833 ...
5,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,"FPIs invest Rs 10,555 cr in equities in Dec so...","Further, capital flows will be dictated by the...",https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96318147,widt...",2022-12-18T08:38:50Z,"After investing over Rs 36,200 crore last mont..."


In [59]:
df['id']=index
df['Polarity'] = np.nan
df['Subjectivity'] = np.nan
df['Score'] = np.nan
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,id,Polarity,Subjectivity,Score
1,"{'id': 'the-times-of-india', 'name': 'The Time...",Reuters,Glittering gold gives markets some Christmas c...,MSCI's broadest index of Asia-Pacific shares o...,https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96386750,widt...",2022-12-21T03:19:42Z,SINGAPORE: Asian stocks were trying get into a...,1,NaN,NaN,NaN
2,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,UK PM Rishi Sunak uses troops to tackle Christ...,There is a wave of winter strikes planned over...,https://economictimes.indiatimes.com/news/inte...,"https://img.etimg.com/thumb/msid-96325937,widt...",2022-12-18T19:23:17Z,British Prime Minister Rishi Sunak on Sunday h...,2,NaN,NaN,NaN
3,"{'id': 'the-times-of-india', 'name': 'The Time...",TNN,Slowing US inflation raises hopes for Santa Cl...,The rally in the global markets could spill ov...,https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96209413,widt...",2022-12-14T02:21:19Z,MUMBAI: An inflation reading in the US that ca...,3,NaN,NaN,NaN
4,"{'id': 'the-times-of-india', 'name': 'The Time...",ET CONTRIBUTORS,Rising Covid cases is polishing gold’s appeal ...,The economic data during the week from the US ...,https://economictimes.indiatimes.com/markets/c...,"https://img.etimg.com/thumb/msid-96477641,widt...",2022-12-24T10:17:12Z,Gold prices reached a six-month high of $1833 ...,4,NaN,NaN,NaN
5,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,"FPIs invest Rs 10,555 cr in equities in Dec so...","Further, capital flows will be dictated by the...",https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96318147,widt...",2022-12-18T08:38:50Z,"After investing over Rs 36,200 crore last mont...",5,NaN,NaN,NaN


In [60]:

from textblob import TextBlob
ListPolarity=[]
ListSubjectivity=[]
ListScore=[]

for id in range(1,101,1):  #for each row in our df dataframe
    newsText = df.description[id]
    if newsText:
        soup = BeautifulSoup(newsText,"lxml") #create a BeautifulSoup object from our HTML news article
        sentA = TextBlob(soup.get_text()) #pass the text only article to TextBlob to anaylse
        ListPolarity.append(sentA.sentiment.polarity) #write sentiment polarity back to list
        ListSubjectivity.append(sentA.sentiment.subjectivity) #write sentiment subjectivity score back to df
    newsText = df.title[id]
    if newsText:
        soup = BeautifulSoup(newsText,"lxml") #create a BeautifulSoup object from our HTML news article
        sentB = TextBlob(soup.get_text()) #pass the text only article to TextBlob to anaylse
        if (sentA.sentiment.polarity >= 0.05  and sentB.sentiment.polarity >= 0.05): 
            score = 'positive'
        elif  (-.05 < sentA.sentiment.polarity < 0.05 and -.05 < sentB.sentiment.polarity < 0.05):
            score = 'neutral'
        elif  (-.05 < sentA.sentiment.polarity < 0.05 and sentB.sentiment.polarity>0.05):
            score = 'neutral'
        elif  ( sentA.sentiment.polarity > 0.05 and -.05 < sentB.sentiment.polarity < 0.05):
            score = 'neutral'
        else:
            score = 'negative'
            
        ListScore.append(score) #write score back to df
      
df.head()


,source,author,title,description,url,urlToImage,publishedAt,content,id,Polarity,Subjectivity,Score
1,"{'id': 'the-times-of-india', 'name': 'The Time...",Reuters,Glittering gold gives markets some Christmas c...,MSCI's broadest index of Asia-Pacific shares o...,https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96386750,widt...",2022-12-21T03:19:42Z,SINGAPORE: Asian stocks were trying get into a...,1,NaN,NaN,NaN
2,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,UK PM Rishi Sunak uses troops to tackle Christ...,There is a wave of winter strikes planned over...,https://economictimes.indiatimes.com/news/inte...,"https://img.etimg.com/thumb/msid-96325937,widt...",2022-12-18T19:23:17Z,British Prime Minister Rishi Sunak on Sunday h...,2,NaN,NaN,NaN
3,"{'id': 'the-times-of-india', 'name': 'The Time...",TNN,Slowing US inflation raises hopes for Santa Cl...,The rally in the global markets could spill ov...,https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96209413,widt...",2022-12-14T02:21:19Z,MUMBAI: An inflation reading in the US that ca...,3,NaN,NaN,NaN
4,"{'id': 'the-times-of-india', 'name': 'The Time...",ET CONTRIBUTORS,Rising Covid cases is polishing gold’s appeal ...,The economic data during the week from the US ...,https://economictimes.indiatimes.com/markets/c...,"https://img.etimg.com/thumb/msid-96477641,widt...",2022-12-24T10:17:12Z,Gold prices reached a six-month high of $1833 ...,4,NaN,NaN,NaN
5,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,"FPIs invest Rs 10,555 cr in equities in Dec so...","Further, capital flows will be dictated by the...",https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96318147,widt...",2022-12-18T08:38:50Z,"After investing over Rs 36,200 crore last mont...",5,NaN,NaN,NaN


In [61]:
if len(ListPolarity)<100:
  a=100-len(ListPolarity)
  for i in range(a):
    ListPolarity.append("NaN")
if len(ListSubjectivity)<100:
  a=100-len(ListSubjectivity)
  for i in range(a):
    ListSubjectivity.append("NaN")
if len(ListScore)<100:
  a=100-len(ListScore)
  for i in range(a):
    ListScore.append("NaN")


df['Polarity'] = ListPolarity
df['Subjectivity'] = ListSubjectivity
df['Score'] = ListScore
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,id,Polarity,Subjectivity,Score
1,"{'id': 'the-times-of-india', 'name': 'The Time...",Reuters,Glittering gold gives markets some Christmas c...,MSCI's broadest index of Asia-Pacific shares o...,https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96386750,widt...",2022-12-21T03:19:42Z,SINGAPORE: Asian stocks were trying get into a...,1,0.111111,0.447222,neutral
2,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,UK PM Rishi Sunak uses troops to tackle Christ...,There is a wave of winter strikes planned over...,https://economictimes.indiatimes.com/news/inte...,"https://img.etimg.com/thumb/msid-96325937,widt...",2022-12-18T19:23:17Z,British Prime Minister Rishi Sunak on Sunday h...,2,0.100000,0.200000,neutral
3,"{'id': 'the-times-of-india', 'name': 'The Time...",TNN,Slowing US inflation raises hopes for Santa Cl...,The rally in the global markets could spill ov...,https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96209413,widt...",2022-12-14T02:21:19Z,MUMBAI: An inflation reading in the US that ca...,3,0.000000,0.000000,neutral
4,"{'id': 'the-times-of-india', 'name': 'The Time...",ET CONTRIBUTORS,Rising Covid cases is polishing gold’s appeal ...,The economic data during the week from the US ...,https://economictimes.indiatimes.com/markets/c...,"https://img.etimg.com/thumb/msid-96477641,widt...",2022-12-24T10:17:12Z,Gold prices reached a six-month high of $1833 ...,4,0.051111,0.319722,positive
5,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,"FPIs invest Rs 10,555 cr in equities in Dec so...","Further, capital flows will be dictated by the...",https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-96318147,widt...",2022-12-18T08:38:50Z,"After investing over Rs 36,200 crore last mont...",5,0.055556,0.277778,positive


Looking at our dataframe we can now see 3 new columns on the right, *Polarity*, *Subjectivity* and *Score*. As we have seen *Polarity* is the actual sentiment polarity returned from **TextBlob** (ranging from -1(negative) to +1(positive), *Subjectivity* is a measure (ranging from 0 to 1) where 0 is very objective and 1 is very subjective, and *Score* is simply a Positive, Negative or Neutral rating based on the strength of the polarities. 



In [62]:
df.to_csv('final.csv') #saving the dataframe as a csv file

### Results and Conclusion

We made our sentiment analysis model to evaluate news articles based on their title and description.

The score value is calculated such that it is negative if either the title or description of an article is negative, this is to make our model more sensitive to nagative articles.

We can further use a NLP model to classify the articles into their respective categories. I have done the same for YouTube Videos using their title and description here: https://team7cic.co